In [181]:
import pandas as pd

filmes = pd.read_csv("data/movies.csv")
notas = pd.read_csv("data/ratings.csv")
filmes = filmes.set_index("movieId")

notas_medias = notas.groupby("movieId").mean()["rating"]
filmes["rating_mean"] = notas_medias

total_votos = notas["movieId"].value_counts()
filmes["rating_quant"] = total_votos

filmes.sort_values("rating_quant", ascending = False).head(10)

,title,genres,rating_mean,rating_quant
movieId,,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,4.164134,329.0
318,"Shawshank Redemption, The (1994)",Crime|Drama,4.429022,317.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.197068,307.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,4.161290,279.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,4.192446,278.0
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,4.231076,251.0
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,3.750000,238.0
110,Braveheart (1995),Action|Drama|War,4.031646,237.0
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,3.970982,224.0


In [311]:
from sklearn.metrics.pairwise import cosine_similarity

user_ratings = notas[notas["userId"] == 1]
all_ratings = notas.merge(filmes, on = "movieId")
all_ratings = all_ratings[["userId", "movieId", "rating"]]
all_ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [313]:
user_ratings = all_ratings.pivot_table("rating", index = "userId", columns = "movieId")
user_ratings = user_ratings.dropna(thresh=10, axis=1).fillna(0)
user_ratings

movieId,1,2,3,5,6,7,9,10,11,12,...,166461,166528,166643,168250,168252,174055,176371,177765,179819,187593
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,2.5,0.0,0.0,2.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [296]:
item_similarity_df = user_ratings.corr(method = "pearson")
item_similarity_df

movieId,1,2,3,5,6,7,9,10,11,12,...,166461,166528,166643,168250,168252,174055,176371,177765,179819,187593
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.231327,0.173213,0.192474,0.192686,0.143743,0.177245,0.183382,0.172799,0.159352,...,-0.003950,0.070499,0.111588,0.022451,0.066439,0.059796,0.054642,-0.013607,0.053028,0.022160
2,0.231327,1.000000,0.191945,0.200526,0.158341,0.127569,-0.021045,0.285086,0.217090,0.115290,...,0.047793,0.167779,0.049701,0.036534,0.195111,0.060834,0.150281,-0.020637,0.113709,0.118390
3,0.173213,0.191945,1.000000,0.370171,0.196442,0.351513,0.275812,0.136916,0.174251,0.168038,...,-0.035086,0.000094,0.018855,-0.031249,-0.000959,0.044769,0.033825,-0.039794,0.026881,-0.039206
5,0.192474,0.200526,0.370171,1.000000,0.215503,0.429890,0.308085,0.110833,0.201002,0.173630,...,0.029139,0.006434,-0.005243,-0.004671,-0.022762,-0.003536,0.000280,-0.038865,-0.000983,-0.038291
6,0.192686,0.158341,0.196442,0.215503,1.000000,0.148109,0.167909,0.251343,0.182082,0.115893,...,-0.052982,0.106208,0.006448,0.038345,0.123248,0.078585,0.119709,-0.060090,0.015800,0.038239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174055,0.059796,0.060834,0.044769,-0.003536,0.078585,0.041336,0.014197,-0.009848,0.042380,0.017650,...,0.096152,0.184902,0.282894,0.341528,0.325772,1.000000,0.689737,0.127251,0.284834,0.128020
176371,0.054642,0.150281,0.033825,0.000280,0.119709,0.029589,-0.001716,0.032775,0.041955,0.083686,...,0.132310,0.434742,0.362638,0.368046,0.476659,0.689737,1.000000,0.169473,0.495133,0.382127
177765,-0.013607,-0.020637,-0.039794,-0.038865,-0.060090,-0.040824,-0.021576,-0.031616,-0.035557,-0.021954,...,0.237951,0.143826,0.085723,0.171565,0.062448,0.127251,0.169473,1.000000,0.362601,0.345148


In [283]:
user_movies = notas.merge(filmes, on = "movieId").query("userId == 2")
user_movies = user_movies.set_index("movieId")
user_movies

,userId,rating,timestamp,title,genres,rating_mean,rating_quant
movieId,,,,,,,
318,2,3.0,1445714835,"Shawshank Redemption, The (1994)",Crime|Drama,4.429022,317.0
333,2,4.0,1445715029,Tommy Boy (1995),Comedy,3.780000,50.0
1704,2,4.5,1445715228,Good Will Hunting (1997),Drama|Romance,4.078014,141.0
3578,2,4.0,1445714885,Gladiator (2000),Action|Adventure|Drama,3.938235,170.0
6874,2,4.0,1445714952,Kill Bill: Vol. 1 (2003),Action|Crime|Thriller,3.961832,131.0
8798,2,3.5,1445714960,Collateral (2004),Action|Crime|Drama|Thriller,3.761364,44.0
46970,2,4.0,1445715013,Talladega Nights: The Ballad of Ricky Bobby (2...,Action|Comedy,3.250000,28.0
48516,2,4.0,1445715064,"Departed, The (2006)",Crime|Drama|Thriller,4.252336,107.0
58559,2,4.5,1445715141,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,4.238255,149.0


In [310]:
import pandas as pd

def get_similar_movies(movie_id, user_rating):
    similar_score = item_similarity_df[movie_id] * (user_rating - 2.5)
    similar_score = similar_score.sort_values(ascending = False)

    return similar_score


def recommend(user_id):
    all_ratings = notas.merge(filmes, on = "movieId")
    user_movies = all_ratings[all_ratings["userId"] == user_id]
    user_movies = user_movies[user_movies["rating_quant"] > 10]
    user_movies = user_movies[["movieId", "rating"]]
    
    similar_movies = pd.DataFrame()
    for movie, rating in user_movies.values:
        similar_movies = pd.concat([similar_movies, pd.DataFrame([get_similar_movies(movie, rating)])], ignore_index=True)

    similar_movies = similar_movies.sum().sort_values(ascending = False)
    return filmes.loc[similar_movies.index]["title"].head(10)
    
recommend(4)

movieId
919                   Wizard of Oz, The (1939)
930                           Notorious (1946)
2997               Being John Malkovich (1999)
1136    Monty Python and the Holy Grail (1975)
2174                        Beetlejuice (1988)
1719               Sweet Hereafter, The (1997)
1079               Fish Called Wanda, A (1988)
1179                      Grifters, The (1990)
912                          Casablanca (1942)
910                    Some Like It Hot (1959)
Name: title, dtype: object